# Tokenizer

## Step 1. Convert text to tokens

In [1]:
with open("dataset/the-verdict.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()
print(f"Total number of characters: {len(raw_text)}")
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [2]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [token.strip() for token in preprocessed if token.strip()]
print(f"Length of tokens: {len(preprocessed)}")
print(preprocessed[:30])

Length of tokens: 4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Step 2. Convert Token to Token ID

In [3]:
all_words = sorted(set(preprocessed))
print(f"Number of unique tokens: {len(all_words)}")

Number of unique tokens: 1130


In [4]:
vocabulary = {token:integer for integer, token in enumerate(all_words)}

In [5]:
for i, t in enumerate(vocabulary.items()):
    print(i, t)
    if i >= 10:
        break

0 ('!', 0)
1 ('"', 1)
2 ("'", 2)
3 ('(', 3)
4 (')', 4)
5 (',', 5)
6 ('--', 6)
7 ('.', 7)
8 (':', 8)
9 (';', 9)
10 ('?', 10)


## Tokenizer Class

In [6]:
class SimpleTokenizerV1:
    def __init__(self, vocabulary):
        self.str_to_int = vocabulary
        self.int_to_str = {i:s for s,i in vocabulary.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [token.strip() for token in preprocessed if token.strip()]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join(self.int_to_str[id] for id in ids)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [7]:
# Test the v1 class methods
tokenizer = SimpleTokenizerV1(vocabulary)
text = """It's the last he painted"""
ids = tokenizer.encode(text)
print(ids)
text = tokenizer.decode(ids)
print(text)

[56, 2, 850, 988, 602, 533, 746]
It' s the last he painted


In [8]:
# Special Context Token
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|unk|>", "<|endoftext|>"])
vocabulary = {token:integer for integer,token in enumerate(all_tokens)}
print(f"Length of Speical Context Tokens: {len(all_tokens)}")

Length of Speical Context Tokens: 1132


In [9]:
for i, item in enumerate(list(vocabulary.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|unk|>', 1130)
('<|endoftext|>', 1131)


In [10]:
class SimpleTokenizerV2:
    def __init__(self, vocabulary):
        self.str_to_int = vocabulary
        self.int_to_str = {i:s for s,i in vocabulary.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [token.strip() for token in preprocessed if token.strip()]
        preprocessed = [token if token in self.str_to_int else "<|unk|>" for token in preprocessed]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join(self.int_to_str[id] for id in ids)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [11]:
# Test the v2 class methods
tokenizer = SimpleTokenizerV2(vocabulary)
text1 = """Hello, do you like tea?"""
text2 = """In the sunlight terraces of the palace"""
text = "<|endoftext|>".join((text1, text2))
print(text)
ids = tokenizer.encode(text)
print(ids)
text = tokenizer.decode(ids)
print(text)

Hello, do you like tea?<|endoftext|>In the sunlight terraces of the palace
[1130, 5, 355, 1126, 628, 975, 10, 1130, 988, 1130, 984, 722, 988, 1130]
<|unk|>, do you like tea? <|unk|> the <|unk|> terraces of the <|unk|>


## Byte Pair Tokenizer

In [12]:
! poetry add tiktoken

The following packages are already present in the pyproject.toml and will be skipped:

  - tiktoken

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [13]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.8.0


In [14]:
tokenizer = tiktoken.get_encoding("gpt2")

In [15]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [16]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [17]:
integers = tokenizer.encode("Akwirw ier")
print(integers)
strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


# Creating Input-Target Pairs

In [1]:
import tiktoken

with open("dataset/the-verdict.txt", "r") as file:
    raw_text = file.read()
tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)
print(f"Length of encoded text: {len(encoded_text)}")

Length of encoded text: 5145


In [2]:
encoded_sample = encoded_text[:50]

In [3]:
context_size = 4
x = encoded_sample[:context_size]
y = encoded_sample[1:context_size+1]
print(f"x is {x}")
print(f"y is {y}")

x is [40, 367, 2885, 1464]
y is [367, 2885, 1464, 1807]


In [4]:
for i in range(1, context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(f"{context} --> {desired}")

[40] --> 367
[40, 367] --> 2885
[40, 367, 2885] --> 1464
[40, 367, 2885, 1464] --> 1807


In [5]:
for i in range(1, context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(f"{tokenizer.decode(context)} --> {tokenizer.decode([desired])}")

I -->  H
I H --> AD
I HAD -->  always
I HAD always -->  thought


In [12]:
! poetry add torch

Using version ^2.5.1 for torch

Updating dependencies
Resolving dependencies... (1.6s)

Package operations: 19 installs, 0 updates, 0 removals

  - Installing nvidia-nvjitlink-cu12 (12.4.127): Pending...
  - Installing nvidia-nvjitlink-cu12 (12.4.127): Installing...
  - Installing nvidia-nvjitlink-cu12 (12.4.127)
  - Installing filelock (3.16.1): Pending...
  - Installing mpmath (1.3.0): Pending...
  - Installing nvidia-cublas-cu12 (12.4.5.8): Pending...
  - Installing nvidia-cusparse-cu12 (12.3.1.170): Pending...
  - Installing nvidia-cublas-cu12 (12.4.5.8): Pending...
  - Installing nvidia-cusparse-cu12 (12.3.1.170): Pending...
  - Installing mpmath (1.3.0): Installing...
  - Installing nvidia-cublas-cu12 (12.4.5.8): Pending...
  - Installing nvidia-cusparse-cu12 (12.3.1.170): Pending...
  - Installing nvidia-cublas-cu12 (12.4.5.8): Pending...
  - Installing nvidia-cusparse-cu12 (12.3.1.170): Pending...
  - Installing mpmath (1.3.0)
  - Installing nvidia-cublas-cu12 (12.4.5.8): Pendi

In [1]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

/home/akash/.cache/pypoetry/virtualenvs/llm-JVV2FBb5-py3.12/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [3]:
with open("dataset/the-verdict.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()

In [6]:
import torch
import tiktoken

print("PyTorch version:", torch.__version__)
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.5.1+cu124
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [7]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [8]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


# CREATING TOKEN EMBEDDINGS

In [9]:
input_ids = torch.tensor([2, 3, 5, 1])

In [10]:
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [11]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [12]:
print(embedding_layer(torch.tensor([3])))


tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [13]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)
